In [5]:
from timm.models.layers import  trunc_normal_,_assert,PatchEmbed
import torch.nn as nn
import torch,math
from einops import rearrange
from thop import profile

#setr解码头
class SETR_up_decoder(nn.Module):
    '''
    
    '''
    def __init__(self, in_channels, out_channels, features=[512, 256, 128, 64]):
        super().__init__()
        self.decoder_1 = nn.Sequential(
                    nn.Conv2d(in_channels, features[0], 3, padding=1),
                    nn.BatchNorm2d(features[0]),
                    nn.ReLU(inplace=True),
                    nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
                )
        self.decoder_2 = nn.Sequential(
                    nn.Conv2d(features[0], features[1], 3, padding=1),
                    nn.BatchNorm2d(features[1]),
                    nn.ReLU(inplace=True),
                    nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
                )
        self.decoder_3 = nn.Sequential(
            nn.Conv2d(features[1], features[2], 3, padding=1),
            nn.BatchNorm2d(features[2]),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=4, mode="bilinear", align_corners=True)
        )
        self.decoder_4 = nn.Sequential(
            nn.Conv2d(features[2], features[3], 3, padding=1),
            nn.BatchNorm2d(features[3]),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        )
        self.final_out = nn.Conv2d(features[-1], out_channels, 3, padding=1)
    def forward(self, x):
        if len(x.shape)==3:
            b,l,c = x.shape
            h = w = int(math.sqrt(l))
            x= rearrange(x,' b (h w) c->b c h w',h=h,w=w)
        x = self.decoder_1(x)
        x = self.decoder_2(x)
        x = self.decoder_3(x)
        x = self.decoder_4(x)
        x = self.final_out(x)
        return x

x = torch.rand(1, 256, 1024)

net = SETR_up_decoder(in_channels=1024, out_channels=2,)
y = net(x)
print(y.shape)

torch.Size([1, 2, 512, 512])


In [6]:
#测试模型输出
import torch,numpy,sys,os,tqdm,timm
from PIL import Image
sys.path.append('D:\software\Code\code-file\pytorch-template')
import numpy as np
from PIL import Image
from torchvision import transforms
Image.MAX_IMAGE_PIXELS = None

strides = 512

#路径设置
image_file = r'D:\software\code\code-file\image\mydata\root\2'
checkpoint_pth = r'D:\software\code\code-file\pytorch-template\saved\models\10_4-timm_model_FX-mobilevitv2_200-MyDataLoader_512-focal_loss-ExponentialLR-Adam-batch_size6-test1\1004_165210\checkpoint-epoch80.pth'
save_file = r'D:\software\code\code-file\image\mydata\root_pre\2_mobilevitv2_200'
if not os.path.exists(save_file):
    os.makedirs(save_file)

net = model.timm_test()
checkpoint = torch.load(checkpoint_pth,map_location='cuda')
state_dict = checkpoint['state_dict']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.load_state_dict(state_dict,False)
net = net.to(device)
net.eval()

to_tensor = transforms.ToTensor()

for image in tqdm.tqdm(os.listdir(image_file)):
    iamge_path = os.path.join(image_file,image)
    save_path = os.path.join(save_file,image)
    image = Image.open(iamge_path).convert("RGB")
    image = np.array(image)
    h,w,c  =  image.shape
    h_n = h//strides
    w_n = w//strides
    h = h_n*strides
    w = w_n*strides

    pre_result = np.empty((h,w))
    for i in range(h_n):
        for j in range(w_n):
            x = strides*i
            y = strides*j

            sub_image = image[x:x + strides , y:y + strides]
            sub_image_T = to_tensor(sub_image).unsqueeze(dim=0).to(device)
            pre = net(sub_image_T)
            pre = torch.squeeze(torch.argmax(pre, dim=1))
            pre = pre.cpu().numpy()
            pre_result[x:x + strides , y:y + strides] = pre
    pre_image = Image.fromarray((pre_result*255).astype(np.uint8))
    pre_image.save(save_path)


100%|██████████| 103/103 [43:04<00:00, 25.09s/it]


In [2]:
#测试模型输出
import torch,numpy,sys,os,tqdm,timm,model
from PIL import Image
sys.path.append('D:\software\Code\code-file\pytorch-template')
import numpy as np
from PIL import Image
from torchvision import transforms
Image.MAX_IMAGE_PIXELS = None


#路径设置
image_path = r'D:\software\code\code-file\image\mydata\root_data2\imgs\test\48_3_2.jpg'
checkpoint_pth = r'D:\software\code\code-file\pytorch-template\saved\models\10_4-timm_model_FX-mobilevitv2_200-MyDataLoader_512-focal_loss-ExponentialLR-Adam-batch_size6-test1\1004_165210\checkpoint-epoch80.pth'

net = model.timm_test()
checkpoint = torch.load(checkpoint_pth,map_location='cuda')
state_dict = checkpoint['state_dict']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.load_state_dict(state_dict,False)
net = net.to(device)
net.eval()
     

with torch.no_grad():
    image = Image.open(image_path)
    to_tensor = transforms.ToTensor()
    image = to_tensor(image).unsqueeze(dim=0).to(device)
    output = net(image)
    pred = torch.squeeze(torch.argmax(output, dim=1))
    img = pred.cpu().numpy()
    num = numpy.sum(img==1)
    print(num)
    img = numpy.where(img==1,255,0)
    img = Image.fromarray(numpy.uint8(img))
    img.save(r"D:\31890\Desktop\1.png")

0
